In [7]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import warnings

from scipy import ndimage
from skimage import filters

from PIL import Image

import utils

warnings.simplefilter('ignore')

### Silicon thickness
Pristine thickness:           Average around 12 microns, but up to 25 microns in particularly thick locations (like in many of the areas that form horizontal, interfacial cracks)\
Lithiated thickness:          Average around 30 microns, but up to 45 microns in particularly thick locations\
De-lithiated thickness:        Average around 22 microns, but up to 40 microns in particularly thick locations\
Re-lithiated thickness:        Average around 25 microns, but up to 50 microns in particularly thick locations

Vertical crack thicknesses (the cracks between each silicon conglomerate after de-lithiation)\
De-lithiated width:          Anywhere from 1 to 15 microns, hard to gather an average but I would say around 3-5 microns\
De-lithiated height:          Same as de-lithiated silicon thicknesses (22 to 40 microns tall)\
Re-lithiated width:          Thinner even where not close up, average around 2-3 microns\
Re-lithiated height:          Cracks close up closer to SSE first, anywhere from 5 to 20 microns tall remaining

Horizontal crack thicknesses (the cracks between certain silicon conglomerates and the solid-state electrolyte (SSE) after de-lithiation)\
De-lithiated width:          Unique to each crack, but can be anywhere from 20 to 50 microns wide\
De-lithiated height:          Very thin, 1-3 microns\
Re-lithiated width:          If the crack still exists, similar dimensions to after de-lithiation\
Re-lithiated height:          If the crack still exists, similar dimensions to after de-lithiation but slightly thinner due to compression from counter electrode (so 1-2 microns max)

In [3]:
data_dir = '/home/molel/OneDrive/PhD/Data/silicon_xct'

In [4]:
subfolders = ['horizonal-cross-sections', 'vertical-cross-sections', 'segmented-crack-volumes']

In [5]:
ignore_phase = 255
silicon_phase = 2
void_phase = 0
ycom = 965
xcom = 520
px_thresh = 254

In [18]:
delith_data_path = os.path.join(data_dir, 'segmented-crack-volumes', 'De-lith segmented crack volume (top down).gif')
utils.make_dir_if_missing('crack-volumes/delithiated')

with tf.io.gfile.GFile(delith_data_path, 'rb') as f:
    video = tf.io.decode_gif(f.read())
    data_delith = np.array(video)
    # remove scale
    data_delith[:, :, 1600:, :] = 255
    

for i in range(data_delith.shape[0]):
    arr = np.copy(data_delith[i, :, :, 0])
    coords_si = np.where(arr > px_thresh)
    coords_void = np.where(np.less_equal(arr, px_thresh))
    seg = np.zeros(arr.shape, dtype=np.uint8)
    seg[coords_void] = void_phase
    seg[coords_si] = silicon_phase
    coords = np.asarray(np.where(arr > -1)).T
    for x, y in coords:
        if ((x - xcom) ** 2 + (y - ycom) ** 2) ** 0.5 > xcom:
            seg[x, y] = ignore_phase
    img = Image.fromarray(seg[:, 425:1500])
    img.save(os.path.join('crack-volumes/delithiated', str(i).zfill(3) + ".tif"))

In [19]:
lith_data_path = os.path.join(data_dir, 'segmented-crack-volumes', 'Re-lith segmented crack volume (top-down).gif')
utils.make_dir_if_missing('crack-volumes/lithiated')
with tf.io.gfile.GFile(lith_data_path, 'rb') as f:
    video = tf.io.decode_gif(f.read())
    data_lith = np.array(video)
    # remove scale
    data_lith[:, :, 1600:, :] = 255

for i in range(data_lith.shape[0]):
    arr = np.copy(data_lith[i, :, :, 0])
    coords_si = np.where(arr > px_thresh)
    coords_void = np.where(np.less_equal(arr, px_thresh))
    seg = np.zeros(arr.shape, dtype=np.uint8)
    seg[coords_void] = void_phase
    seg[coords_si] = silicon_phase
    coords = np.asarray(np.where(arr > -1)).T
    for x, y in coords:
        if ((x - xcom) ** 2 + (y - ycom) ** 2) ** 0.5 > xcom:
            seg[x, y] = ignore_phase
    img = Image.fromarray(seg[:, 425:1500])
    img.save(os.path.join('crack-volumes/lithiated', str(i).zfill(3) + ".tif"))